In [113]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

In [114]:
x_data = np.array([[2,1],[2,3],[4,1],[5,6],[7,7],[8,9],[9,10],[6,7],[6,10],[2,8],[7,5]])

kf = KFold(n_splits=5)

In [115]:
for train_index, test_index in kf.split(x_data):
    print(train_index)
    print(test_index)

[ 3  4  5  6  7  8  9 10]
[0 1 2]
[ 0  1  2  5  6  7  8  9 10]
[3 4]
[ 0  1  2  3  4  7  8  9 10]
[5 6]
[ 0  1  2  3  4  5  6  9 10]
[7 8]
[0 1 2 3 4 5 6 7 8]
[ 9 10]


### K-Fold 교차검증 -> 회귀 문제에서 사용
- 학습 data와 테스트 data를 n_splits개의 세트로 나누어 검증하는 방법
- data set이 적을 때 train data를 늘리기 위해 사용
- 여러 개의 train, test data 짝으로 검증과정을 거침

In [116]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [117]:
x_data = np.array([[2,1],[2,3],[4,1],[5,6],[7,7],[8,9],[9,10],[6,7],[6,10],[2,8],[7,5]])
y_data = np.array([3,5,9,7,10,2,15,6,4,7,8])

In [118]:
model = LinearRegression()

train_scores = []
test_scores = []

kf = KFold(n_splits=5)
for train_index, test_index in kf.split(x_data):
    x_train = np.array(x_data)[train_index]
    y_train = np.array(y_data)[train_index]
    x_test = np.array(x_data)[test_index]
    y_test = np.array(y_data)[test_index]

    model_kf = LinearRegression()
    model_kf.fit(x_train, y_train)
    score = model_kf.score(x_train, y_train)    # R2
    train_scores.append(score)
    score = model_kf.score(x_test, y_test)
    test_scores.append(score)

train_scores, test_scores

([0.09087045589617082,
  0.14634108316388406,
  0.2596984905781652,
  0.22523882013459684,
  0.35657970453058097],
 [0.33939878660391054,
  -0.28777247567478437,
  0.06722521416975036,
  -9.902863890644971,
  -157.7869721150263])

In [119]:
np.array(train_scores).mean()

0.21574571086067956

In [120]:
np.array(test_scores).mean()

-33.51419689611448

R2값
- -, + 부호는 그래프의 기울기
- 절대값은 data를 잘 설명하는 정도(-1~1 사이의 값을 가짐) : 너무 커지거나 작아지면 data가 쓰레기라는 뜻

In [121]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

In [122]:
model = LinearRegression()
cv_results = cross_validate(model, x_data, y_data)
cv_results

{'fit_time': array([0.0009954 , 0.        , 0.00099564, 0.        , 0.        ]),
 'score_time': array([0.        , 0.        , 0.        , 0.00098872, 0.        ]),
 'test_score': array([ 3.39398787e-01, -2.87772476e-01,  6.72252142e-02, -9.90286389e+00,
        -1.57786972e+02])}

In [123]:
cv_results['test_score'].mean()

-33.51419689611448

In [124]:
df = pd.DataFrame(cv_results)
df = df.sort_values(by='test_score', ascending=False)   # 내림차순
df

,fit_time,score_time,test_score
0,0.000995,0.000000,0.339399
2,0.000996,0.000000,0.067225
1,0.000000,0.000000,-0.287772
3,0.000000,0.000989,-9.902864
4,0.000000,0.000000,-157.786972


In [125]:
model = LinearRegression()
model.fit(x_data, y_data)

LinearRegression()

In [126]:
cvs = cross_val_score(model, x_data, y_data, cv=5)
cvs

array([ 3.39398787e-01, -2.87772476e-01,  6.72252142e-02, -9.90286389e+00,
       -1.57786972e+02])

In [127]:
cvs.mean()

-33.51419689611448

분류

In [128]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [129]:
x_data = np.array([
    [2,1],
    [3,2],
    [3,4],
    [5,5],
    [7,5],
    [2,5],
    [8,9],
    [9,10],
    [6, 12],
    [9,2],
    [6,10],
    [2,4]
])
y_data = np.array([3,5,7,10,12,7,13,13,12,13,12,6])
label = ['A','B','C']

In [132]:
model = LogisticRegression()
cv_results = cross_validate(model, x_data, y_data, return_estimator=True)
# cv_results = cross_validate(model, x_data, y_data, return_estimator=True)
cv_results

ValueError: n_splits=5 cannot be greater than the number of members in each class.

In [ ]:
cv_results['test_score'].mean()

-33.51419689611448

In [ ]:
df = pd.DataFrame(cv_results)
df = df.sort_values(by='test_score', ascending=False)
df

,fit_time,score_time,test_score
0,0.000000,0.0,0.339399
2,0.000000,0.0,0.067225
1,0.000000,0.0,-0.287772
3,0.000998,0.0,-9.902864
4,0.000000,0.0,-157.786972


### 계층적 k-겹 교차검증(stratified k-fold cross validation)
- 분류 모델에 적용
- k-겹 교차검증 모델은 k-fold가 원본 data 집합의 레이블 분포를 학습 및 검증 data set에 제대로 분배하지 못하는 문제를 해결
- target 값(= 레이블/클래스)의 속성값의 개수를 골고루 넣어주게 됨

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
iris = load_iris()

In [ ]:
x = iris.data
y = iris.target

skf = StratifiedKFold(n_splits=5, random_state=30, shuffle=True)

In [ ]:
idx_iter = 0
cv_acc = []

# split으로 반환된 인덱스를 이용해서 학습데이터, 테스트데이터를 정의해줌
for train_index, test_index in skf.split(x,y):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(x_train, y_train)
    pred = model.predict(x_test)

    idx_iter += 1
    acc = np.round(accuracy_score(y_test, pred),4)
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]

    print('{0}번째 교차 검증 정확도 : {1} \n 학습 데이터 크기 : {2} \n 검증 데이터 크기 : {3}'.format(idx_iter,acc,train_size,test_size))
    cv_acc.append(acc)

1번째 교차 검증 정확도 : 0.9333 
 학습 데이터 크기 : 120 
 검증 데이터 크기 : 30
2번째 교차 검증 정확도 : 0.9667 
 학습 데이터 크기 : 120 
 검증 데이터 크기 : 30
3번째 교차 검증 정확도 : 1.0 
 학습 데이터 크기 : 120 
 검증 데이터 크기 : 30
4번째 교차 검증 정확도 : 0.9333 
 학습 데이터 크기 : 120 
 검증 데이터 크기 : 30
5번째 교차 검증 정확도 : 1.0 
 학습 데이터 크기 : 120 
 검증 데이터 크기 : 30


c:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 